In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').option('numFeatures', '10').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(
    featuresCol='features',
    labelCol='label',
    predictionCol='prediction'
)

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

### Train Test Split

In [12]:
all_data = spark.read.format('libsvm').option('numFeatures', '10').load('sample_linear_regression_data.txt')

In [13]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                338|
|   mean| 0.6485659736119701|
| stddev| 10.118023729547717|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [15]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                163|
|   mean|-0.5553006083506477|
| stddev| 10.706469454146688|
|    min|-28.046018037776633|
|    max| 27.111027963108548|
+-------+-------------------+



In [16]:
correct_model = lr.fit(train_data)

In [17]:
test_results = correct_model.evaluate(test_data)

In [18]:
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-27.991275855503922|
|-28.024270045905734|
|-22.663750583140917|
| -23.29352662003837|
|-25.987619982333023|
+-------------------+
only showing top 5 rows



In [19]:
test_results.rootMeanSquaredError

11.02186203941731

In [20]:
unlabeld_data = test_data.select('features')

In [21]:
unlabeld_data.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [22]:
predictions = correct_model.transform(unlabeld_data)

In [23]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.05474218227271...|
|(10,[0,1,2,3,4,5,...|   1.218786617422662|
|(10,[0,1,2,3,4,5,...| -0.8471335071820557|
|(10,[0,1,2,3,4,5,...|   1.861138855872563|
|(10,[0,1,2,3,4,5,...|  5.7755427233743495|
|(10,[0,1,2,3,4,5,...|  1.9646626437107702|
|(10,[0,1,2,3,4,5,...|   2.015914365505433|
|(10,[0,1,2,3,4,5,...| 0.09179275286179622|
|(10,[0,1,2,3,4,5,...|  0.7337636355567213|
|(10,[0,1,2,3,4,5,...|  0.9191515880999007|
|(10,[0,1,2,3,4,5,...|  1.7287758409507648|
|(10,[0,1,2,3,4,5,...| -2.6151450507764533|
|(10,[0,1,2,3,4,5,...|   0.895690722175587|
|(10,[0,1,2,3,4,5,...| -0.3105371100280151|
|(10,[0,1,2,3,4,5,...|-0.03221386530925...|
|(10,[0,1,2,3,4,5,...|   1.070658127697382|
|(10,[0,1,2,3,4,5,...|  2.8113385238877995|
|(10,[0,1,2,3,4,5,...| -2.2576834330528652|
|(10,[0,1,2,3,4,5,...| 0.26402585131700174|
|(10,[0,1,2,3,4,5,...|   1.69955